In [4]:
!pip install nbeats_keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
beatrix-jupyterlab 2023.46.184821 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which

In [6]:
# BASE
# ------------------------------------------------------
import time

import numpy as np
import pandas as pd
import os
import gc
import warnings
from pathlib import Path
#!pip install xgboost
from tqdm import tqdm

import sklearn
from sklearn import linear_model
from sklearn.linear_model  import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit 
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
pd.set_option('display.max_colwidth', None)
import statsmodels.api as sm
import importlib.util

import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX



import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from nbeats_keras.model import NBeatsNet
import tensorflow as tf
from keras import backend as K



from pandas import datetime
from pandas import read_csv
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')
print('config')

config


In [7]:
import os
import importlib.util
#exit() 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == 'data_setup_kaggle.py':
            print('Found data_setup_kaggle.py')
            module_name = 'data_setup_kaggle'
            module_path = os.path.join(dirname, filename)
            
            spec = importlib.util.spec_from_file_location(module_name, module_path)
            data_setup = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(data_setup)
            
            # Execute a function from the imported module and assign its result to 'd'
            d = data_setup.get_data()
            
            # Now you can use the imported module
            # ...
            
            
# Compile the model
def rmsle_loss(y_true, y_pred):

    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = K.square(y_true_log - y_pred_log)
    mean_squared_diff = K.mean(squared_diff, axis=-1)
    rmsle = K.sqrt(mean_squared_diff)
    return rmsle

Found data_setup_kaggle.py
Parent directory: /kaggle
(3029400, 81)
complete


In [8]:
d.head()
#d = d[d['date'].dt.year > 2016]
d.head()




,id,date,store_nbr,family,sales,onpromotion,test/train,city,state,type,cluster,events_Black_Friday,events_Cyber_Monday,events_Dia_de_la_Madre,events_Futbol,events_Terremoto_Manabi,holiday_national_binary,holiday_local_binary,holiday_regional_binary,national_independence,local_cantonizacio,local_fundacion,local_independencia,holiday_national_Batalla_de_Pichincha,holiday_national_Carnaval,holiday_national_Dia_de_Difuntos,holiday_national_Dia_de_la_Madre,holiday_national_Dia_del_Trabajo,holiday_national_Independencia_de_Cuenca,holiday_national_Independencia_de_Guayaquil,holiday_national_Navidad,holiday_national_Primer_Grito_de_Independencia,holiday_national_Primer_dia_del_ano,holiday_national_Viernes_Santo,holiday_regional_Provincializacion_Santa_Elena,holiday_regional_Provincializacion_de_Cotopaxi,holiday_regional_Provincializacion_de_Imbabura,holiday_regional_Provincializacion_de_Santo_Domingo,holiday_local_Cantonizacion_de_Cayambe,holiday_local_Cantonizacion_de_El_Carmen,holiday_local_Cantonizacion_de_Guaranda,holiday_local_Cantonizacion_de_Latacunga,holiday_local_Cantonizacion_de_Libertad,holiday_local_Cantonizacion_de_Quevedo,holiday_local_Cantonizacion_de_Riobamba,holiday_local_Cantonizacion_de_Salinas,holiday_local_Cantonizacion_del_Puyo,holiday_local_Fundacion_de_Ambato,holiday_local_Fundacion_de_Cuenca,holiday_local_Fundacion_de_Esmeraldas,holiday_local_Fundacion_de_Guayaquil,holiday_local_Fundacion_de_Ibarra,holiday_local_Fundacion_de_Loja,holiday_local_Fundacion_de_Machala,holiday_local_Fundacion_de_Manta,holiday_local_Fundacion_de_Quito,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,workday,wageday,dcoilwtico,dcoilwtico_interpolated
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
1,1,2013-01-01,1,BABY CARE,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
2,2,2013-01-01,1,BEAUTY,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
3,3,2013-01-01,1,BEVERAGES,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
4,4,2013-01-01,1,BOOKS,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14


In [9]:
metrics_df = pd.DataFrame(columns=['Model', 'family_encoded', 'cols', 'MAE', 'MSE', 'RMSE','RMSLE', 'R2'])

# preform test train split
cols = ['test/train' ,'sales', 'store_nbr', 'type', 'family', 'date', 'month','day_of_month', 'week_of_year','day_of_week', 'dcoilwtico_interpolated']
#cols = ['test/train' ,'sales', 'family', 'date', 'month','day_of_month', 'week_of_year','day_of_week', 'type', 'dcoilwtico_interpolated']

dataTrainTest = d[cols]

dataTrain = dataTrainTest[dataTrainTest['test/train'] ==  'train'].reset_index(drop=True)
dataTest = dataTrainTest[dataTrainTest['test/train'] ==  'test'].reset_index(drop=True)

dataTrain

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataTrain["family_encoded"] = encoder.fit_transform(dataTrain["family"])
dataTrain.drop('family', axis=1, inplace=True)

dataTrain["type"]
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataTrain["type_encoded"] = encoder.fit_transform(dataTrain["type"])
dataTrain.drop('type', axis=1, inplace=True)

train_ = (dataTrain)
test_ = (dataTest)

#train_, test_ = train_test_split(dataTrain.dropna().reset_index(drop=True), test_size=0.2, random_state=42)

del d
gc.collect()


8

In [10]:
#dataTrain = dataTrain[dataTrain['family_encoded'] == 27 ]
dataTrain

,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded
0,train,0.00,1,2013-01-01,1,1,1,2,93.14,0,3
1,train,0.00,1,2013-01-01,1,1,1,2,93.14,1,3
2,train,0.00,1,2013-01-01,1,1,1,2,93.14,2,3
3,train,0.00,1,2013-01-01,1,1,1,2,93.14,3,3
4,train,0.00,1,2013-01-01,1,1,1,2,93.14,4,3
...,...,...,...,...,...,...,...,...,...,...,...
3000883,train,438.13,9,2017-08-15,8,15,33,2,47.32,28,1
3000884,train,154.55,9,2017-08-15,8,15,33,2,47.32,29,1
3000885,train,2419.73,9,2017-08-15,8,15,33,2,47.32,30,1
3000886,train,121.00,9,2017-08-15,8,15,33,2,47.32,31,1


In [11]:
cols_out = list(dataTrain.columns)
cols_out.append('y_pred')

In [12]:
cols_out

['test/train',
 'sales',
 'store_nbr',
 'date',
 'month',
 'day_of_month',
 'week_of_year',
 'day_of_week',
 'dcoilwtico_interpolated',
 'family_encoded',
 'type_encoded',
 'y_pred']

In [13]:
#dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]
dataTrain['family_encoded'].nunique()

33

In [ ]:
from joblib import Parallel, delayed
import sys

# Define a function to fit SARIMAX model
def fit_sarimax(data, exogenous, so):
    model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=so)
    model_fit = model.fit(disp=False, maxiter=2000, method='nm')
    return model_fit

# Define a helper function to calculate metrics
def calculate_metrics(test_ARIMA, y_pred_ARIMA):
    mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
    mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
    rmse = np.sqrt(mse)
    r2 = r2_score(test_ARIMA, y_pred_ARIMA)
    y_pred_ARIMA[y_pred_ARIMA < 0] = 0
    rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))
    return mae, mse, rmse, rmsle, r2

dataTrain = dataTrain.sort_values(['family_encoded', 'date', 'type_encoded'])
dataTrain_out = pd.DataFrame(columns=cols_out)
metrics_df = pd.DataFrame(columns=['Model', 'cols', 'MAE', 'MSE', 'RMSE', 'RMSLE', 'R2'])

#def process_family(family):
#    global dataTrain
# Define the process_family function
def process_family(family):
    global dataTrain_out, metrics_df, dataTrain
    print(family)
    sys.stdout.flush()
    dataTrainFamily = dataTrain[dataTrain['family_encoded'] == family]
    last_month = dataTrainFamily['date'].max().to_period('M')
    start_date = last_month.start_time + pd.DateOffset(days=1)
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]

    series = train_.copy(deep=True)
    series.set_index('date', inplace=True)
    series.index = series.index.to_period('M')

    test_inner = test_.copy(deep=True)
    test_inner.set_index('date', inplace=True)
    test_inner.index = test_inner.index.to_period('M')

    test_exog = test_inner.drop(['test/train', 'sales'], axis=1)
    exogenous = series.drop(['test/train', 'sales'], axis=1)

    test_ARIMA = test_inner['sales']

    pdqm = [(1, 0, 1, 12)]

    for so in pdqm:
        exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
        #exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
        exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
        #exogenous['month'] = pd.Categorical(exogenous['month'])
        #exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
        #exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
        #exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])

        series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
        test_ARIMA = pd.to_numeric(test_ARIMA, downcast='integer')

        test_exog['dcoilwtico_interpolated'] = test_exog['dcoilwtico_interpolated'].astype('int8')
        #test_exog['family_encoded'] = pd.Categorical(test_exog['family_encoded'])
        test_exog['type_encoded'] = pd.Categorical(test_exog['type_encoded'])
        #test_exog['month'] = pd.Categorical(test_exog['month'])
        #test_exog['day_of_month'] = pd.Categorical(test_exog['day_of_month'])
        #test_exog['week_of_year'] = pd.Categorical(test_exog['week_of_year'])
        #test_exog['day_of_week'] = pd.Categorical(test_exog['day_of_week'])

       # try:
        start_time = time.time()
        #model = fit_sarimax(series['sales'], exogenous, so)
        #model_fit = model.fit()

        model = SARIMAX(series['sales'], order=(1, 0, 4), seasonal_order=(so))
        model_fit = model.fit(disp=False)#, method='nm', max_iter=1000)

        #_pred_ARIMA = model_fit.predict(start=test_inner.index[0].start_time, end=test_inner.index[-1].end_time, exog=test_exog)


                # Calculate the fitting time
        fitting_time = time.time() - start_time
        # Print the fitting time
        print("Fitting time:", fitting_time, "seconds")
        sys.stdout.flush()

        y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)


        y_pred_ARIMA = np.round(y_pred_ARIMA).astype(int)
        mae, mse, rmse, rmsle, r2 = calculate_metrics(test_ARIMA, y_pred_ARIMA)
        sys.stdout.flush()

        result = pd.DataFrame([[str(so), family, exogenous.columns.tolist(), mae, mse, rmse, rmsle, r2]],
                              columns=['Model', 'family_encoded', 'cols', 'MAE', 'MSE', 'RMSE', 'RMSLE', 'R2'])
        #metrics_df = metrics_df.append(result, ignore_index=True)
        
        test_['y_pred'] = y_pred_ARIMA.values
        dataTrain_out = pd.concat([dataTrain_out, test_], ignore_index=True)
        print('Faamily mean ', result['RMSLE'].mean())

        #print('rmsle',rmsle)
        #print('test_[y_pred]',test_['y_pred'])
        #print('dataTrain_out[y_pred]',dataTrain_out['y_pred'])

        sys.stdout.flush()

        del model_fit, y_pred_ARIMA
        gc.collect()
        #except Exception as e:
         #   print(e)

    return (dataTrain_out)





# Use joblib to parallelize the family-wise processing
#dataTrain_result = Parallel(n_jobs=4, prefer='processes')(delayed(process_family)(family) for family in [12, 13, 14, 15])
dataTrain_result = Parallel(n_jobs=4, prefer='processes')(delayed(process_family)(family) for family in dataTrain['family_encoded'].unique())

for data in dataTrain_result:
    dataTrain_out = pd.concat([dataTrain_out, data], ignore_index=True)


    
#Parallel(n_jobs=4, prefer='processes')(delayed(process_family)(family) for family in dataTrain['family_encoded'].unique())


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0
1
2
3


In [ ]:
group = dataTrain_out.groupby('family_encoded')

for g in group:
    df = g[1]  # Access the DataFrame from the tuple using index 1
    #print(df)
    mae, mse, rmse, rmsle, r2 = calculate_metrics((df['sales']).to_numpy(), (df['y_pred']).to_numpy())
    dataTrain_out.loc[dataTrain_out['family_encoded'] ==df['family_encoded'].iloc[0], ['mae', 'mse', 'rmse', 'rmsle', 'r2']] = mae, mse, rmse, rmsle, r2

    print('Family: {}, MAE: {:.2f}, MSE: {:.2f}, RMSE: {:.2f}, RMSLE: {:.2f}, R2: {:.2f}'.format(df['family_encoded'].iloc[0], mae, mse, rmse, rmsle, r2))

dataTrain_out

In [ ]:
dataTrain_out['%_diff'] = dataTrain_out['sales'] / dataTrain_out['y_pred']

In [ ]:
dataTrain_out['%_diff'].mean()

In [ ]:
dataTrain_out['%_diff'].group_by('family_encoded').mean()

In [24]:
dataTrain_out
#metrics_df
#result

,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded,y_pred
0,train,11156.00,44,2017-08-02,8,2,31,3,49.27,12,0,9393
1,train,12494.00,45,2017-08-02,8,2,31,3,49.27,12,0,9241
2,train,10701.00,46,2017-08-02,8,2,31,3,49.27,12,0,8756
3,train,12263.00,47,2017-08-02,8,2,31,3,49.27,12,0,7818
4,train,9945.00,48,2017-08-02,8,2,31,3,49.27,12,0,7693
...,...,...,...,...,...,...,...,...,...,...,...,...
3019,train,40.00,8,2017-08-15,8,15,33,2,47.32,15,3,12
3020,train,38.00,28,2017-08-15,8,15,33,2,47.32,15,4,0
3021,train,33.00,29,2017-08-15,8,15,33,2,47.32,15,4,0
3022,train,32.00,36,2017-08-15,8,15,33,2,47.32,15,4,0


In [ ]:
from joblib import Parallel, delayed

# Define a function to fit SARIMAX model
def fit_sarimax(data, exogenous, so):
    model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=so)
    model_fit = model.fit(disp=False)
    return model_fit

# Define a helper function to calculate metrics
def calculate_metrics(test_ARIMA, y_pred_ARIMA):
    mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
    mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
    rmse = np.sqrt(mse)
    rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))
    r2 = r2_score(test_ARIMA, y_pred_ARIMA)
    return mae, mse, rmse, rmsle, r2

dataTrain = dataTrain.sort_values(['family_encoded', 'date', 'type_encoded'])
dataTrain_out = pd.DataFrame(columns=cols_out)
metrics_df = pd.DataFrame(columns=['Model', 'cols', 'MAE', 'MSE', 'RMSE', 'RMSLE', 'R2'])

#def process_family(family):
#    global dataTrain
def process_family(family):
    global dataTrain_out, metrics_df, dataTrain
    print(family)
    sys.stdout.flush()
    dataTrainFamily = dataTrain[dataTrain['family_encoded'] == family]
    #dataTrainFamily = dataTrainFamily.iloc[int(84875):]#[29229395:-1]
    
    last_month = dataTrainFamily['date'].max().to_period('M')
    start_date = last_month.start_time + pd.DateOffset(days=1)
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]
    
    series = train_.copy(deep=True)
    series.set_index('date', inplace=True)
    series.index = series.index.to_period('M')
    
    test_inner = test_.copy(deep=True)
    test_inner.set_index('date', inplace=True)
    test_inner.index = test_inner.index.to_period('M')
    
    test_exog = test_inner.copy(deep=True)
    test_exog = test_exog.drop(['test/train', 'sales'], axis=1)
    exogenous = series.drop(['test/train', 'sales'], axis=1)
    
    test_ARIMA = test_inner['sales']
    
    pdqm = [(1, 0, 1, 12)]
    
    for so in pdqm:
        exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
        exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
        exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
        exogenous['month'] = pd.Categorical(exogenous['month'])
        exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
        exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
        exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])
        
        series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
        test_ARIMA = pd.to_numeric(test_ARIMA, downcast='integer')
        
        test_exog['dcoilwtico_interpolated'] = test_exog['dcoilwtico_interpolated'].astype('int8')
        test_exog['family_encoded'] = pd.Categorical(test_exog['family_encoded'])
        test_exog['type_encoded'] = pd.Categorical(test_exog['type_encoded'])
        test_exog['month'] = pd.Categorical(test_exog['month'])
        test_exog['day_of_month'] = pd.Categorical(test_exog['day_of_month'])
        test_exog['week_of_year'] = pd.Categorical(test_exog['week_of_year'])
        test_exog['day_of_week'] = pd.Categorical(test_exog['day_of_week'])
        
        memory_bytes = exogenous.memory_usage(deep=True).sum()
        memory_gb = memory_bytes / (1024 ** 3)
        print(f"Size of data: {memory_gb:.2f} GB")
        
        start_time = time.time()
        
        # Fit SARIMAX models in parallel using joblib

        model = SARIMAX(series['sales'], order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit(disp=False)
        del model
        
        # Calculate the fitting time
        fitting_time = time.time() - start_time

        # Print the fitting time
        print("Fitting time:", fitting_time, "seconds")


        y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)

        test_['y_pred_raw'] = y_pred_ARIMA.values

        y_pred_ARIMA[y_pred_ARIMA < 0] = 0


        if (y_pred_ARIMA < 0).any():
            rmsle = 9999999

        else:    
            print((test_ARIMA.shape),(y_pred_ARIMA.shape))
            print(type(test_ARIMA),type(y_pred_ARIMA.shape))
            rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))
            sys.stdout.flush()


        # Evaluate the model
        mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
        mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
        rmse = np.sqrt(mse)

        r2 = r2_score(test_ARIMA, y_pred_ARIMA)

        print(f'MAE: {mae:.2f}')
        print(f'MSE: {mse:.2f}')
        print(f'RMSE: {rmse:.2f}')
        print(f'RMSLE: {rmsle:.2f}')
        print(f'R2 score: {r2:.2f}')
        sys.stdout.flush()

        print(y_pred_ARIMA.values)
        test_['y_pred'] = y_pred_ARIMA.values
        
        dataTrain_out = dataTrain_out.append(test_)

        metrics_df = metrics_df.append({'Model': 'SARIMAX', 'family_encoded':family , 'cols': list(train_.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)
        print('rolling mean RMSLE',metrics_df['RMSLE'].mean())
        del y_pred_ARIMA, test_exog, series, exogenous
        gc.collect()

# Use joblib to parallelize the family-wise processing
Parallel(n_jobs=4, prefer='processes')(delayed(process_family)(family) for family in dataTrain['family_encoded'].unique())

dataTrain_out

# Use joblib to parallelize the family-wise processing
#dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]
#dataTrain
#Parallel(n_jobs=4, prefer='processes')(delayed(process_family)(family) for family in dataTrain['family_encoded'].unique())


In [ ]:
dataTrain['family_encoded'].nunique()

In [ ]:
dataTrain_out


In [ ]:
import multiprocessing as mp
global dataTrain

# Define a function to fit SARIMAX model
def fit_sarimax(data, exogenous, so):
    model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=so)
    model_fit = model.fit(disp=False)
    return model_fit

# Define a helper function to calculate metrics
def calculate_metrics(test_ARIMA, y_pred_ARIMA):
    mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
    mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
    rmse = np.sqrt(mse)
    rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))
    r2 = r2_score(test_ARIMA, y_pred_ARIMA)
    return mae, mse, rmse, rmsle, r2

dataTrain = dataTrain.sort_values(['family_encoded', 'date', 'type_encoded'])
dataTrain_out = pd.DataFrame(columns=cols_out)
metrics_df = pd.DataFrame(columns=['Model', 'cols', 'MAE', 'MSE', 'RMSE', 'RMSLE', 'R2'])

def process_family(dataTrain):
    for family in tqdm(dataTrain['family_encoded'].unique()):
        print(family)
        dataTrainFamily = dataTrain[dataTrain['family_encoded'] == family]
        dataTrainFamily = dataTrainFamily.iloc[int(84875):]#[29229395:-1]

        last_month = dataTrainFamily['date'].max().to_period('M')
        start_date = last_month.start_time + pd.DateOffset(days=1)
        train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
        test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]

        series = train_.copy()
        series.set_index('date', inplace=True)
        series.index = series.index.to_period('M')

        test_inner = test_.copy()
        test_inner.set_index('date', inplace=True)
        test_inner.index = test_inner.index.to_period('M')

        test_exog = test_inner.copy()
        test_exog = test_exog.drop(['test/train', 'sales'], axis=1)
        exogenous = series.drop(['test/train', 'sales'], axis=1)

        test_ARIMA = test_inner['sales']

        pdqm = [(1, 0, 1, 12)]

        for so in pdqm:
            exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
            exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
            exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
            exogenous['month'] = pd.Categorical(exogenous['month'])
            exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
            exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
            exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])

            series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
            test_ARIMA = pd.to_numeric(test_ARIMA, downcast='integer')

            test_exog['dcoilwtico_interpolated'] = test_exog['dcoilwtico_interpolated'].astype('int8')
            test_exog['family_encoded'] = pd.Categorical(test_exog['family_encoded'])
            test_exog['type_encoded'] = pd.Categorical(test_exog['type_encoded'])
            test_exog['month'] = pd.Categorical(test_exog['month'])
            test_exog['day_of_month'] = pd.Categorical(test_exog['day_of_month'])
            test_exog['week_of_year'] = pd.Categorical(test_exog['week_of_year'])
            test_exog['day_of_week'] = pd.Categorical(test_exog['day_of_week'])

            memory_bytes = exogenous.memory_usage(deep=True).sum()
            memory_gb = memory_bytes / (1024 ** 3)
            print(f"Size of data: {memory_gb:.2f} GB")

            start_time = time.time()

            # Create a pool of processes
            sys.stdout.flush()

            pool = mp.Pool()

            # Fit SARIMAX models in parallel
            results = [pool.apply_async(fit_sarimax, args=(series['sales'], exogenous, so)) for _ in range(4)]

            # Get the fitted models
            model_fits = [result.get() for result in results]

            # Close the pool of processes
            pool.close()

            # Wait for all processes to finish
            pool.join()

            # Calculate the fitting time
            fitting_time = time.time() - start_time
            print("Fitting time:", fitting_time, "seconds")
            sys.stdout.flush()

            for i, model_fit in enumerate(model_fits):
                y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog=test_exog)
                test_['y_pred_raw'] = y_pred_ARIMA.values
                y_pred_ARIMA[y_pred_ARIMA < 0] = 0

                if (y_pred_ARIMA < 0).any():
                    rmsle = 9999999
                else:
                    rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

                mae, mse, rmse, rmsle, r2 = calculate_metrics(test_ARIMA, y_pred_ARIMA)

                print(f'MAE: {mae:.2f}')
                print(f'MSE: {mse:.2f}')
                print(f'RMSE: {rmse:.2f}')
                print(f'RMSLE: {rmsle:.2f}')
                print(f'R2 score: {r2:.2f}')

                test_['y_pred'] = y_pred_ARIMA.values
                dataTrain_out = dataTrain_out.append(test_)
                metrics_df = metrics_df.append({'Model': 'SARIMAX', 'family_encoded': family, 'cols': list(train_.columns), 
                                                'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)

                print('rolling mean RMSLE', metrics_df['RMSLE'].mean())
                del y_pred_ARIMA, test_exog, series, exogenous
                gc.collect()

# Use multiprocessing to parallelize the family-wise processing
pool = mp.Pool()
#dataTrain = dataTrain[dataTrain['family_encoded'] == 12 ]
dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]

pool.map(process_family, dataTrain)
pool.close()
pool.join()

dataTrain_out


In [ ]:
dataTrain_out

In [ ]:
dataTrain = dataTrain.sort_values(['family_encoded','date', 'type_encoded'])

dataTrain_out = pd.DataFrame(columns = cols_out)
metrics_df = pd.DataFrame(columns=['Model', 'cols', 'MAE', 'MSE', 'RMSE','RMSLE', 'R2'])
global y_pred_ARIMA

for family in tqdm(dataTrain['family_encoded'].unique()):
    #y_pred_ARIMA
    print(family)
    dataTrainFamily =  dataTrain[dataTrain['family_encoded'] == family ]

    #dataTrainFamily = dataTrainFamily.iloc[int(84875):]#[29229395:-1]
        
    # Find the last month in the 'date' column
    last_month = dataTrainFamily['date'].max().to_period('M')

    # Calculate the start date for the test data
    start_date = last_month.start_time + pd.DateOffset(days=1)

    # Filter the DataFrame based on the dynamic timeframe
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]

    series = train_.copy()
    #series = series.sort_values(['date','family_encoded'])
    series.set_index('date', inplace=True)
    series
    series.index = series.index.to_period('M')


    test_inner = test_.copy()
    #test_inner = test_inner.sort_values(['date','family_encoded'])
    test_inner.set_index('date', inplace=True)
    test_inner
    test_inner.index = test_inner.index.to_period('M')

    test_exog = test_inner.copy()
    test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)
    exogenous = series.drop(['test/train',  'sales'] , axis=1)

    test_ARIMA = test_inner['sales']



    def fit_sarimax(data,exogenous):
        # Fit SARIMAX model to the subset of data
        so = (1, 0, 1, 12)
        model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit()
        return model_fit

    pdqm = [
    (1, 0, 1, 12)
    ]

    for so in pdqm:
        #for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
        #print(f"pdqm {so}:")

        exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
        # Convert categorical columns to categorical data type
        exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
        exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
        exogenous['month'] = pd.Categorical(exogenous['month'])
        exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
        exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
        exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])

        series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
        test_ARIMA = pd.to_numeric(test_ARIMA, downcast='integer')

        test_exog['dcoilwtico_interpolated'] = test_exog['dcoilwtico_interpolated'].astype('int8')
        # Convert categorical columns to categorical data type
        test_exog['family_encoded'] = pd.Categorical(test_exog['family_encoded'])
        test_exog['type_encoded'] = pd.Categorical(test_exog['type_encoded'])
        test_exog['month'] = pd.Categorical(test_exog['month'])
        test_exog['day_of_month'] = pd.Categorical(test_exog['day_of_month'])
        test_exog['week_of_year'] = pd.Categorical(test_exog['week_of_year'])
        test_exog['day_of_week'] = pd.Categorical(test_exog['day_of_week'])        
    


        # Get the memory usage of each column
        memory_usage = exogenous.memory_usage(deep=True)
        #print("\nMemory Usage:")
        #print(memory_usage)
        
        # Calculate memory usage in bytes
        memory_bytes = exogenous.memory_usage(deep=True).sum()
        # Convert to gigabytes (GB)
        memory_gb = memory_bytes / (1024**3)
        # Print the size in GB
        print(f"Size of data: {memory_gb:.2f} GB")

        start_time = time.time()
        #model = ARIMA(series, order=(1,0,4), exog=exogenous )


        model = SARIMAX(series['sales'], order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit(disp=False)
        del model
        
        # Calculate the fitting time
        fitting_time = time.time() - start_time

        # Print the fitting time
        print("Fitting time:", fitting_time, "seconds")


        y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)

        test_['y_pred_raw'] = y_pred_ARIMA.values

        y_pred_ARIMA[y_pred_ARIMA < 0] = 0


        if (y_pred_ARIMA < 0).any():
            rmsle = 9999999

        else:    
            #print((test_ARIMA.shape),(y_pred_ARIMA.shape))
            #print(type(test_ARIMA),type(y_pred_ARIMA.shape))
            rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

        # Evaluate the model
        mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
        mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
        rmse = np.sqrt(mse)

        r2 = r2_score(test_ARIMA, y_pred_ARIMA)

        print(f'MAE: {mae:.2f}')
        print(f'MSE: {mse:.2f}')
        print(f'RMSE: {rmse:.2f}')
        print(f'RMSLE: {rmsle:.2f}')
        print(f'R2 score: {r2:.2f}')


        test_['y_pred'] = y_pred_ARIMA.values
        
        dataTrain_out = dataTrain_out.append(test_)

        metrics_df = metrics_df.append({'Model': 'SARIMAX', 'family_encoded':family , 'cols': list(train_.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)
        print('rolling mean RMSLE',metrics_df['RMSLE'].mean())
        del y_pred_ARIMA, test_exog, series, exogenous
        gc.collect()
        
        #print(dataTrainFamily)
dataTrain_out



In [ ]:
print('Mean RMSLE',metrics_df['RMSLE'].mean())

In [ ]:
metrics_df = metrics_df.sort_values(by='RMSLE')
metrics_df

In [ ]:
metrics_df.to_csv('metrics_df.csv',index=False)

In [ ]:
dataTrain_out.to_csv('dataTrain_out.csv',index=False)

In [11]:

#!pip install nbeats-keras
from sklearn.preprocessing import LabelEncoder
from nbeats_keras.model import NBeatsNet
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
from keras import backend as K

# Define custom RMSLE metric
def rmsle(y_true, y_pred):
    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = K.square(y_true_log - y_pred_log)
    mean_squared_diff = K.mean(squared_diff, axis=-1)
    rmsle = K.sqrt(mean_squared_diff)
    return rmsle


# Find the last month in the 'date' column
last_month = dataTrain['date'].max().to_period('M')

# Calculate the start date for the test data
start_date = last_month.start_time + pd.DateOffset(days=1)

# Filter the DataFrame based on the dynamic timeframe
train_ = dataTrain[dataTrain['date'] < start_date]
test_ = dataTrain[dataTrain['date'] >= start_date]

train_.shape

In [ ]:
train_

In [ ]:
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from nbeats_keras.model import NBeatsNet
import tensorflow as tf


# Assuming your data is stored in variables X and y
X = train_.drop(['sales', 'test/train', 'date'], axis=1).values
y = train_['sales'].values

# Normalize the input features
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_normalized = (X - X_mean) / X_std
print(X_normalized.shape)

# Define the NBeatsNet model
input_shape = X_normalized.shape[1]
output_shape = 1  # Assuming you have a single output target
backcast_length = 10  # Length of the backcast sequence
forecast_length = 1  # Length of the forecast sequence
stack_layers = 30  # Number of stack layers
stack_width = 512  # Width of each stack
output_activation = 'linear'  # Activation function for output layer

# Define the model architecture
model = NBeatsNet(input_dim=1, output_dim=1, stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
                  nb_blocks_per_stack=3, forecast_length=forecast_length, backcast_length=backcast_length,
                  thetas_dim=(4, 4), share_weights_in_stack=False, hidden_layer_units=64)

# Compile the model
model.compile(optimizer=Adam(), loss='mse')

X_normalized = #tf.convert_to_tensor(?????), dtype=tf.float32)

# Train the model
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_normalized, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


In [12]:
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from nbeats_keras.model import NBeatsNet
import tensorflow as tf
from keras import backend as K


dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]
#dataTrain = dataTrain[dataTrain['family_encoded'].isin([12])]

test_appened = pd.DataFrame(columns = list(dataTrain.columns))

# Compile the model
def rmsle_loss(y_true, y_pred):

    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = K.square(y_true_log - y_pred_log)
    mean_squared_diff = K.mean(squared_diff, axis=-1)
    rmsle = K.sqrt(mean_squared_diff)
    return rmsle


models = []

for family in tqdm(dataTrain['family_encoded'].unique()):
    print(family)
    dataTrainFamily = dataTrain[dataTrain['family_encoded'] == family]
        
    # Find the last month in the 'date' column
    last_month = dataTrainFamily['date'].max().to_period('M')

    # Calculate the start date for the test data
    start_date = last_month.start_time + pd.DateOffset(days=1)

    # Filter the DataFrame based on the dynamic timeframe
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]
    
    test_appened = test_appened.append(test_)

    # Assuming your data is stored in variables X and y
    X = train_.drop(['sales', 'test/train', 'date'], axis=1).values
    y = train_['sales'].values
    
    # Define the model architecture
    model = NBeatsNet(input_dim=dataTrain.drop(['sales', 'test/train', 'date'], axis=1).shape[1], output_dim=1, stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
                      nb_blocks_per_stack=3, forecast_length=1, backcast_length=1,
                      thetas_dim=(4, 4), share_weights_in_stack=False, hidden_layer_units=64)

    # Compile the model with RMSLE loss
    model.compile(loss=rmsle_loss, optimizer='adam')#model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmsle])


    # Normalize the input features
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_normalized = (X - X_mean) / X_std

    # Define the NBeatsNet model
    input_shape = X_normalized.shape[1]
    print(input_shape)
    output_shape = 1  # Assuming you have a single output target
    backcast_length =  train_['family_encoded'].nunique()  # Length of the backcast sequence
    forecast_length = 1  # Length of the forecast sequence
    stack_layers = 30  # Number of stack layers
    stack_width = 512  # Width of each stack
    output_activation = 'linear'  # Activation function for output layer




    # Reshape the input data
    num_samples = X_normalized.shape[0]
    print(X_normalized.shape)
    X_reshaped = np.reshape(X_normalized, ( int(num_samples / train_['family_encoded'].nunique()), train_['family_encoded'].nunique(), input_shape))
    X_tensor = tf.convert_to_tensor(X_reshaped, dtype=tf.float32)

    # Train the model
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    model.fit(X_tensor, y, epochs=3, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
    models.append(model)

  0%|          | 0/4 [00:00<?, ?it/s]

12
8
(11502, 8)
Epoch 1/3
288/288 [==============================] - 28s 35ms/step - loss: 1.5084 - val_loss: 1.0200
Epoch 2/3
288/288 [==============================] - 9s 30ms/step - loss: 0.7368 - val_loss: 0.9465
Epoch 3/3
288/288 [==============================] - 8s 29ms/step - loss: 0.7313 - val_loss: 0.9644


 25%|██▌       | 1/4 [00:47<02:22, 47.58s/it]

13
8
(11502, 8)
Epoch 1/3
288/288 [==============================] - 28s 34ms/step - loss: 1.2715 - val_loss: 1.0609
Epoch 2/3
288/288 [==============================] - 8s 28ms/step - loss: 1.1867 - val_loss: 1.1148
Epoch 3/3
288/288 [==============================] - 8s 29ms/step - loss: 1.1862 - val_loss: 1.1593


 50%|█████     | 2/4 [01:34<01:34, 47.18s/it]

14
8
(11502, 8)
Epoch 1/3
288/288 [==============================] - 27s 34ms/step - loss: 0.6277 - val_loss: 0.6540
Epoch 2/3
288/288 [==============================] - 8s 29ms/step - loss: 0.6235 - val_loss: 0.6695
Epoch 3/3
288/288 [==============================] - 8s 28ms/step - loss: 0.6241 - val_loss: 0.6640


 75%|███████▌  | 3/4 [02:28<00:50, 50.49s/it]

15
8
(11502, 8)
Epoch 1/3
288/288 [==============================] - 28s 34ms/step - loss: 0.9323 - val_loss: 0.8016
Epoch 2/3
288/288 [==============================] - 8s 27ms/step - loss: 0.7875 - val_loss: 0.8248
Epoch 3/3
288/288 [==============================] - 8s 29ms/step - loss: 0.7856 - val_loss: 0.8767


100%|██████████| 4/4 [03:24<00:00, 51.02s/it]


In [40]:
y_test_all = [] 
y_pred_all = []
for model, family in zip(models, test_appened['family_encoded'].unique()):
    # Assuming your test data is stored in variables X_test and y_test
    test = test_appened[test_appened['family_encoded'] == family]

    X_test = test.drop(['sales', 'test/train', 'date'], axis=1).values
    y_test = test['sales'].values

    # Normalize the input features using the same mean and standard deviation from training data
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_std_nonzero = X_std.copy()
    X_std_nonzero[X_std_nonzero == 0] = 1  # Replace zero standard deviations with 1 to avoid division by zero
    X_test_normalized = (X_test - X_mean) / X_std_nonzero


    # Reshape the input data
    num_samples_test = X_test_normalized.shape[0]
    X_test_reshaped = np.reshape(X_test_normalized, (int(num_samples_test / backcast_length), backcast_length, input_shape))
    X_tensor = tf.convert_to_tensor(X_reshaped, dtype=tf.float32)

    # Predict on the test data
    y_pred = model.predict(X_tensor)

    # Perform any necessary post-processing on the predicted values

    # Evaluate the model's performance on the test data
    loss = rmsle_loss(y_test, y_pred)
    print(f"Test Loss: {np.mean(loss)}")
    #print(f"Test Loss: {loss}")
    y_test_all.append(y_test)
    y_pred_all.append(y_pred)
    #y_test_all = np.append(y_test_all, y_test, axis=0)
    #y_pred_all = np.append(y_pred_all, y_pred, axis=0)



loss = rmsle_loss(list(y_test_all), list(y_pred_all))
y_teslosst_all = np.reshape(loss, (-1, loss.shape[-1]))
print(f" Total Test Loss: {np.mean(y_teslosst_all)}")
print(f"Test Loss: {loss}")

360/360 [==============================] - 3s 9ms/step
Test Loss: 0.5242944359779358
360/360 [==============================] - 3s 9ms/step
Test Loss: 1.1064798831939697
360/360 [==============================] - 3s 9ms/step
Test Loss: 0.6291805505752563
360/360 [==============================] - 3s 9ms/step
Test Loss: 0.7509424090385437
 Total Test Loss: 3.210923671722412
Test Loss: [[[0.53891844 5.2966223  7.474173   4.950731  ]
  [0.53891844 5.2966223  7.474173   4.950731  ]
  [0.53891844 5.2966223  7.474173   4.950731  ]
  ...
  [0.51569295 5.456182   7.636218   5.111582  ]
  [0.51569295 5.456182   7.636218   5.111582  ]
  [0.51569295 5.456182   7.636218   5.111582  ]]

 [[5.7647886  1.1074593  1.9658301  1.0098543 ]
  [5.7647886  1.1074593  1.9658301  1.0098543 ]
  [5.7647886  1.1074593  1.9658301  1.0098543 ]
  ...
  [5.7655716  1.107737   1.9650855  1.0103912 ]
  [5.7655716  1.107737   1.9650855  1.0103912 ]
  [5.7655716  1.107737   1.9650855  1.0103912 ]]

 [[7.647302   2.50352

In [1]:
print(f" Total Test Loss: {np.mean(y_teslosst_all)}")
print(f"Test Loss: {y_teslosst_all}")
y_teslosst_all

NameError: name 'np' is not defined

In [43]:
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from nbeats_keras.model import NBeatsNet
import tensorflow as tf
from keras import backend as K


dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]
#dataTrain = dataTrain[dataTrain['family_encoded'].isin([12])]

test_appened = pd.DataFrame(columns = list(dataTrain.columns))

# Define the model architecture
model = NBeatsNet(input_dim=dataTrain.drop(['sales', 'test/train', 'date'], axis=1).shape[1], output_dim=1, stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
                  nb_blocks_per_stack=3, forecast_length=1, backcast_length=1,
                  thetas_dim=(4, 4), share_weights_in_stack=False, hidden_layer_units=64)

# Compile the model
def rmsle_loss(y_true, y_pred):

    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = K.square(y_true_log - y_pred_log)
    mean_squared_diff = K.mean(squared_diff, axis=-1)
    rmsle = K.sqrt(mean_squared_diff)
    return rmsle

# Compile the model with RMSLE loss
model.compile(loss=rmsle_loss, optimizer='adam')#model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmsle])


for family in tqdm(dataTrain['family_encoded'].unique()):
    print(family)
    dataTrainFamily = dataTrain[dataTrain['family_encoded'] == family]
        
    # Find the last month in the 'date' column
    last_month = dataTrainFamily['date'].max().to_period('M')

    # Calculate the start date for the test data
    start_date = last_month.start_time + pd.DateOffset(days=1)

    # Filter the DataFrame based on the dynamic timeframe
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]
    
    test_appened = test_appened.append(test_)

    # Assuming your data is stored in variables X and y
    X = train_.drop(['sales', 'test/train', 'date'], axis=1).values
    y = train_['sales'].values


    # Normalize the input features
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_normalized = (X - X_mean) / X_std

    # Define the NBeatsNet model
    input_shape = X_normalized.shape[1]
    print(input_shape)
    output_shape = 1  # Assuming you have a single output target
    backcast_length =  train_['family_encoded'].nunique()  # Length of the backcast sequence
    forecast_length = 1  # Length of the forecast sequence
    stack_layers = 30  # Number of stack layers
    stack_width = 512  # Width of each stack
    output_activation = 'linear'  # Activation function for output layer




    # Reshape the input data
    num_samples = X_normalized.shape[0]
    print(X_normalized.shape)
    X_reshaped = np.reshape(X_normalized, ( int(num_samples / train_['family_encoded'].nunique()), train_['family_encoded'].nunique(), input_shape))
    X_tensor = tf.convert_to_tensor(X_reshaped, dtype=tf.float32)

    # Train the model
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    model.fit(X_tensor, y, epochs=3, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

  0%|          | 0/4 [00:00<?, ?it/s]

12
8
(11502, 8)
Epoch 1/3


  0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [10]:
test_appened

,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded
7041,train,3247.00,1,2017-08-02,8,2,31,3,49.27,12,3
7074,train,2177.00,1,2017-08-03,8,3,31,4,49.40,12,3
7107,train,2778.00,1,2017-08-04,8,4,31,5,49.37,12,3
7140,train,2034.00,1,2017-08-05,8,5,31,6,49.50,12,3
7173,train,1025.00,1,2017-08-06,8,6,31,7,49.44,12,3
...,...,...,...,...,...,...,...,...,...,...,...
404364,train,75.00,9,2017-08-11,8,11,32,5,48.58,15,1
404397,train,57.00,9,2017-08-12,8,12,32,6,48.40,15,1
404430,train,58.00,9,2017-08-13,8,13,32,7,48.00,15,1
404463,train,41.00,9,2017-08-14,8,14,33,1,47.72,15,1


In [11]:




# Assuming your test data is stored in variables X_test and y_test
X_test = test_appened.drop(['sales', 'test/train', 'date'], axis=1).values
y_test = test_appened['sales'].values

# Normalize the input features using the same mean and standard deviation from training data
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_std_nonzero = X_std.copy()
X_std_nonzero[X_std_nonzero == 0] = 1  # Replace zero standard deviations with 1 to avoid division by zero
X_test_normalized = (X_test - X_mean) / X_std_nonzero


# Reshape the input data
num_samples_test = X_test_normalized.shape[0]
X_test_reshaped = np.reshape(X_test_normalized, (int(num_samples_test / backcast_length), backcast_length, input_shape))
X_tensor = tf.convert_to_tensor(X_reshaped, dtype=tf.float32)

# Predict on the test data
y_pred = model.predict(X_tensor)

# Perform any necessary post-processing on the predicted values

# Evaluate the model's performance on the test data
loss = rmsle_loss(y_test, y_pred)
print(f"Test Loss: {np.mean(loss)}")
print(f"Test Loss: {loss}")


360/360 [==============================] - 5s 9ms/step
Test Loss: 2.9450061321258545
Test Loss: [[2.948526 ]
 [2.948526 ]
 [2.948526 ]
 ...
 [2.9431324]
 [2.9431324]
 [2.9431324]]


In [ ]:
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from nbeats_keras.model import NBeatsNet
import tensorflow as tf

#dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]
dataTrain = dataTrain[dataTrain['family_encoded'].isin([12])]

# Find the last month in the 'date' column
last_month = dataTrain['date'].max().to_period('M')

# Calculate the start date for the test data
start_date = last_month.start_time + pd.DateOffset(days=1)

# Filter the DataFrame based on the dynamic timeframe
train_ = dataTrain[dataTrain['date'] < start_date]
test_ = dataTrain[dataTrain['date'] >= start_date]


# Assuming your data is stored in variables X and y
X = train_.drop(['sales', 'test/train', 'date'], axis=1).values
y = train_['sales'].values


# Normalize the input features
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_normalized = (X - X_mean) / X_std

# Define the NBeatsNet model
input_shape = X_normalized.shape[1]
output_shape = 1  # Assuming you have a single output target
backcast_length =  train_['family_encoded'].nunique()  # Length of the backcast sequence
forecast_length = 1  # Length of the forecast sequence
stack_layers = 30  # Number of stack layers
stack_width = 512  # Width of each stack
output_activation = 'linear'  # Activation function for output layer

# Define the model architecture
model = NBeatsNet(input_dim=8, output_dim=1, stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
                  nb_blocks_per_stack=3, forecast_length=forecast_length, backcast_length=backcast_length,
                  thetas_dim=(4, 4), share_weights_in_stack=False, hidden_layer_units=64)

# Compile the model
def rmsle_loss(y_true, y_pred):
    
    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = K.square(y_true_log - y_pred_log)
    mean_squared_diff = K.mean(squared_diff, axis=-1)
    rmsle = K.sqrt(mean_squared_diff)
    return rmsle

# Compile the model with RMSLE loss
model.compile(loss=rmsle_loss, optimizer='adam')#model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmsle])


# Reshape the input data
num_samples = X_normalized.shape[0]
print(X_normalized.shape)
X_reshaped = np.reshape(X_normalized, ( int(num_samples / train_['family_encoded'].nunique()), train_['family_encoded'].nunique(), input_shape))
X_tensor = tf.convert_to_tensor(X_reshaped, dtype=tf.float32)

# Train the model
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_tensor, y, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

In [33]:
dataTrain = dataTrain[dataTrain['family_encoded'].isin([12, 13, 14, 15])]
#dataTrain = dataTrain[dataTrain['family_encoded'].isin([12])]

# Find the last month in the 'date' column
last_month = dataTrain['date'].max().to_period('M')

# Calculate the start date for the test data
start_date = last_month.start_time + pd.DateOffset(days=1)

# Filter the DataFrame based on the dynamic timeframe
train_ = dataTrain[dataTrain['date'] < start_date]
test_ = dataTrain[dataTrain['date'] >= start_date]


In [14]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

train_data = train_.drop(['sales', 'test/train', 'date'], axis=1).values
train_targets = train_['sales'].values
train_data.shape

# Define the input shape based on your data dimensions
input_shape = (train_data.shape[1], 1)
print (input_shape)
print (train_data.shape)

# Create the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=input_shape))  # 64 is the number of LSTM units (you can adjust this value)
model.add(Dense(1))  # Output layer with a single neuron for regression (adjust as needed)

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Adjust optimizer and loss function as per your task

# Train the model
model.fit(train_data, train_targets, epochs=10, batch_size=256)

# Once trained, you can use the model to make predictions on new data
test_data = np.random.random((10, 8))  # Replace this with your actual test data
test_data = (test_.shape[1], 1)
#predictions = model.predict(test_data)

# Print the predicted values
#print(predictions)

(8, 1)
(11502, 8)
Epoch 1/10
45/45 [==============================] - 3s 4ms/step - loss: 31079334.0000
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 31000152.0000
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 30948670.0000
Epoch 4/10
45/45 [==============================] - 0s 4ms/step - loss: 30913628.0000
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 30881818.0000
Epoch 6/10
45/45 [==============================] - 0s 4ms/step - loss: 30851282.0000
Epoch 7/10
45/45 [==============================] - 0s 6ms/step - loss: 30821594.0000
Epoch 8/10
45/45 [==============================] - 0s 4ms/step - loss: 30792430.0000
Epoch 9/10
45/45 [==============================] - 0s 4ms/step - loss: 30763614.0000
Epoch 10/10
45/45 [==============================] - 0s 4ms/step - loss: 30735070.0000


In [43]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Assuming your training data is stored in a variable named 'data'
train_data = train_.drop(['sales', 'test/train', 'date'], axis=1).values
train_targets = train_['sales'].values # Replace this with your actual training targets

print (input_shape)
print (train_data.shape)

# Reshape the input data to have a 3D shape
#train_data_reshaped = pd.wide_to_long(train_data, stubnames='', i='index', j='timestep', sep='').reset_index()
#train_data_reshaped = pd.pivot(train_data_reshaped, index='player', columns='team', values='points')

train_data = train_data.reshape((int(train_data.shape[0]/ train_['family_encoded'].nunique()), train_data.shape[1],  train_['family_encoded'].nunique()))
print(train_data.shape)
print((train_data.shape[1], train_data.shape[2]))

# Create the LSTM model
model = Sequential()
model.add(LSTM(16, input_shape=(train_data.shape[1], train_data.shape[2])))
model.add(Dense(1))

# Compile and train the model as before
model.compile(optimizer='adam', loss='mse')
print(model.summary())
model.fit(train_data, train_targets, epochs=1, batch_size=32)

(8, 1)
(46008, 8)
(11502, 8, 4)
(8, 4)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 16)                1344      
                                                                 
 dense_16 (Dense)            (None, 4)                 68        
                                                                 
Total params: 1,412
Trainable params: 1,412
Non-trainable params: 0
_________________________________________________________________
None


ValueError: Data cardinality is ambiguous:
  x sizes: 11502
  y sizes: 46008
Make sure all arrays contain the same number of samples.

In [70]:

# Creating a dictionary
d = {
    'A':[1,1,1,1,1,2],
}

# Creating a dataframe
df = pd.DataFrame(d)
df['B'] = df['A']

# Display original DataFrame
print("Original DataFrame:\n",df,"\n")

# Creating a new column
df['C'] = df['B'].diff()

# Filter result
res = df[df['C'] != 0].index

# Display result
print("Result:\n",res[1:],"\n")

Original DataFrame:
    A  B
0  1  1
1  1  1
2  1  1
3  1  1
4  1  1
5  2  2 

Result:
 Int64Index([5], dtype='int64') 



In [80]:
dataTrain.sort_values(by=['date','family_encoded']).reset_index(drop=True)

,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded
0,train,0.00,1,2017-01-01,1,1,52,7,53.05,12,3
1,train,0.00,10,2017-01-01,1,1,52,7,53.05,12,2
2,train,0.00,11,2017-01-01,1,1,52,7,53.05,12,1
3,train,0.00,12,2017-01-01,1,1,52,7,53.05,12,2
4,train,0.00,13,2017-01-01,1,1,52,7,53.05,12,2
...,...,...,...,...,...,...,...,...,...,...,...
49027,train,24.00,54,2017-08-15,8,15,33,2,47.32,15,2
49028,train,24.00,6,2017-08-15,8,15,33,2,47.32,15,3
49029,train,42.00,7,2017-08-15,8,15,33,2,47.32,15,3
49030,train,40.00,8,2017-08-15,8,15,33,2,47.32,15,3


In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [11]:
# Assuming your training data is stored in a variable named 'data'
dataTrain_sorted = dataTrain.sort_values(by=['date','family_encoded']).reset_index(drop=True)

dataTrain_sorted['family_encoded_B'] = dataTrain_sorted['family_encoded']

dataTrain_sorted['change_point'] = dataTrain_sorted['family_encoded_B'].diff()

change_points = dataTrain_sorted[dataTrain_sorted['change_point'] != 0].index

change_points = change_points[1:]

change_points

dataTrain_sorted['family_encoded']


0          0
1          0
2          0
3          0
4          0
          ..
404509    32
404510    32
404511    32
404512    32
404513    32
Name: family_encoded, Length: 404514, dtype: int64

In [30]:
# Assuming your training data is stored in a variable named 'data'
dataTrain_sorted = dataTrain.sort_values(by=['date','family_encoded']).reset_index(drop=True)

# Use shift to compare current values with the previous values in the column
dataTrain_sorted['change_point'] = dataTrain_sorted['family_encoded'].ne(dataTrain_sorted['family_encoded'].shift())

# Find the row numbers where the change occurs
change_points = dataTrain_sorted[dataTrain_sorted['change_point']].index
change_points = change_points[1:]


train_data = dataTrain_sorted.drop(['sales', 'test/train', 'date' ,'change_point'], axis=1).values
train_targets = dataTrain_sorted['sales'].values # Replace this with your actual training targets

input_shape = (train_data.shape[1], 1)
print(train_data.shape)
print(input_shape)
print(train_targets.shape)

# Normalize the input features
X_mean = np.mean(train_data, axis=0)
X_std = np.std(train_data, axis=0)
train_data = (train_data - X_mean) / X_std


# Split the data into separate sequences based on category or date boundaries
sequences = []  # List to store individual sequences
current_sequence = []  # Temporary list to store the current sequence
targets = []

reset_points = change_points  # List of indices where the reset should happen

for i, data_point in enumerate(train_data):
    #print(i, data_point.shape)
    current_sequence.append(data_point)
    
    if i in reset_points:
        #print('in')
        sequences.append(current_sequence)
        current_sequence = []

batch_size = 32  # Set the desired batch size
input_dim = train_data.shape[1]
        
# Create the LSTM model
model = Sequential()
model.add(LSTM(16, input_shape=(input_shape), stateful=True, batch_input_shape=(batch_size, input_dim, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

print((batch_size, input_dim, 1))
old_cp = 0
i=0
# Train the model with resetting of internal state
for sequence, cp in zip(sequences, change_points):
    # Convert the sequence to numpy array format
    sequence = np.array(sequence)
    print('loop',sequence.shape, old_cp,cp)

    
    # Reset the internal state of the LSTM model
    
    # Train the model with the current sequence
    if i == 0:
        model.fit(sequence, train_targets[old_cp:cp+1], epochs=1,  validation_split=0.1, batch_size=32,  shuffle=False)
    else:
        model.fit(sequence, train_targets[old_cp:cp], epochs=1,  validation_split=0.1, batch_size=32,  shuffle=False)
    i+=1
    old_cp = cp
    model.reset_states()



(404514, 8)
(8, 1)
(404514,)
(32, 8, 1)
loop (55, 8) 0 54
1/2 [==============>...............] - ETA: 14s - loss: 0.7799

InvalidArgumentError: Graph execution error:

Invalid input_h shape: [1,32,16] [1,17,16]
	 [[{{node CudnnRNN}}]]
	 [[sequential_14/lstm_14/PartitionedCall]] [Op:__inference_train_function_11830]

In [ ]:
data_shape = train_.drop(['sales','test/train', 'date'], axis=1).values.shape
print("Data shape:", data_shape)
train = train_.drop(['sales','test/train', 'date'], axis=1).shape

backcast_length = 365

In [ ]:
# Define the model architecture
model = NBeatsNet(input_dim=1, output_dim=1, stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
                  nb_blocks_per_stack=3, forecast_length=5, backcast_length=10,
                  thetas_dim=(4, 4), share_weights_in_stack=False, hidden_layer_units=64)

# Compile the model
def rmsle_loss(y_true, y_pred):
    
    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = K.square(y_true_log - y_pred_log)
    mean_squared_diff = K.mean(squared_diff, axis=-1)
    rmsle = K.sqrt(mean_squared_diff)
    return rmsle

# Compile the model with RMSLE loss
model.compile(loss=rmsle_loss, optimizer='adam')#model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmsle])

# Set training parameters
batch_size = 64
epochs = 50
early_stopping_patience = 10

In [ ]:
import tensorflow as tf

# Prepare the input and target data
#X_train = dataTrain['sales'].values.reshape(-1, 1)
y_train = train_['sales'].values.reshape(-1, 1)
#X_train = dataTrain.drop('sales', axis=1).values.reshape(-1, 1)
num_features = train_.drop(['sales','test/train', 'date'], axis=1).shape[1]
num_features

print (365 ,num_features)
X_train = tf.convert_to_tensor(train_.drop(['sales','test/train', 'date'], axis=1).values.reshape((-1, 365, num_features)), dtype=tf.float32)



# Create an early stopping callback
early_stopping = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

# Fit the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping])

In [ ]:


# Define the parameters
input_dim = train_.drop(['sales', 'test/train', 'date'], axis=1).shape[1]
backcast_length = 5
forecast_length = 5

# Reduce the data size to fit the desired shape
num_samples = train_.shape[0] // (backcast_length + forecast_length)
num_samples = min(num_samples, 3067)  # Set the desired number of samples here

# Reshape the data
train_data = train_.drop(['sales', 'test/train', 'date'], axis=1).values[:num_samples * (backcast_length + forecast_length)]
train_data = train_data.reshape((num_samples, backcast_length + forecast_length, input_dim))

# Convert the reshaped data to TensorFlow tensor
X_train = tf.convert_to_tensor(train_data, dtype=tf.float32)

# Create the N-BEATS model
model = NBeatsNet(input_dim=input_dim, backcast_length=backcast_length, forecast_length=forecast_length,
                  stack_types=(NBeatsNet.GENERIC_BLOCK,), nb_blocks_per_stack=3, thetas_dim=(4,), share_weights_in_stack=False)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model to the data
model.fit(X_train, X_train, epochs=10, batch_size=32)

In [ ]:
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from nbeats_keras.model import NBeatsNet

backcast_length 

# Define the NBeatsNet model
model = NBeatsNet(input_dim=1, output_dim=1, stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK),
                  nb_blocks_per_stack=3, forecast_length=7, backcast_length=backcast_length,
                  thetas_dim=(4, 4), share_weights_in_stack=False, hidden_layer_units=64)

# Define the RMSLE loss function
def rmsle_loss(y_true, y_pred):
    epsilon = 1e-7  # Small constant to prevent division by zero
    y_true = tf.maximum(y_true, epsilon)  # Ensure y_true is always positive
    y_pred = tf.maximum(y_pred, epsilon)  # Ensure y_pred is always positive
    y_true_log = tf.math.log1p(y_true)
    y_pred_log = tf.math.log1p(y_pred)
    squared_diff = tf.square(y_true_log - y_pred_log)
    mean_squared_diff = tf.reduce_mean(squared_diff, axis=-1)
    rmsle = tf.sqrt(mean_squared_diff)
    return rmsle



# Define the parameters
input_dim = train_.shape[1]
backcast_length = 10
forecast_length = 5

# Reshape the data
num_samples = train_.shape[0]
total_length = backcast_length + forecast_length
print(train_.shape)
print(num_samples, total_length, input_dim)

train_data = np.reshape(train_, (num_samples, total_length, input_dim))
print(train_.shape())


In [ ]:
train_.drop(['sales','test/train', 'date'], axis=1).shape

In [ ]:
# Prepare the input data for prediction
X_test = test_['sales'].values.reshape(-1, 1)

# Make predictions
predictions = model.predict(X_test)

# Add the predictions to the test data
test_data['predicted_target'] = predictions.flatten()